In [0]:
eclosures = {}
def epsilon_closure(state,transition):
    global eclosures
    if eclosures.get(state):
        return eclosures[state]
    else:
        eclosures[state] = set([state])
        cur_state = state
        if transition.get((cur_state,'E')):
            free_states = set(transition[(cur_state,'E')])
            eclosures[state] = eclosures[state].union(free_states)
            for f in free_states:
                e = epsilon_closure(f,transition)
                eclosures[state] = eclosures[state].union(e)
        return eclosures[state]

def all_eclosures(states,transition):
    for state in states:
        epsilon_closure(state,transition)
        
def enfa_to_nfa(enfa):
    states = enfa["states"]
    transition = enfa["transition"]
    #all_eclosures(states,transition) #this initializes the dic eclosures
    for state in states:
        
        eclosure = epsilon_closure(state, transition)
        for t in enfa["symbols"]:
            new_transition = []
            for st in eclosure:
                if transition.get((st,t)):
                    X = transition[(st,t)]
                    for x in X:
                        new_transition += list(epsilon_closure(x,transition))
            if(len(new_transition)):
                new_transition = list(set(new_transition))
                transition[(state,t)] = new_transition
    for state in states:
        transition.pop((state,'E'),None)
        
    return enfa #nfa
    

In [0]:
def to_string(lis):
    lis = [str(x) for x in lis]
    lis = ",".join(lis)
    return lis

def nfa_to_dfa(nfa):
    start = nfa["start"]
    symbols = nfa["symbols"]
    transition = {}
    all_states = [[start]]
    queue = []
    queue.append([start])
    while(len(queue)):
        states = queue[0]
        queue = queue[1:]
        for symbol in symbols:
            out = []
            for state in states:
                if nfa["transition"].get((state,symbol)):
                    out += nfa["transition"][(state,symbol)]
            string_rep = to_string(states)
            if transition.get((string_rep,symbol)):
                pass
            else:
                if len(out):
                    out = list(set(out))
                    transition[(string_rep,symbol)] = to_string(out)
                    if out not in all_states:
                        all_states.append(out)
                        queue.append(out)
                else:
                    #X represents dead state
                    transition[(string_rep,symbol)]='X'
                    deadstate=1

                    
    #final = [to_string(state) for state in all_states if nfa["final"] in state]
    final = [to_string(state) for state in all_states 
             if any([f in state for f in nfa["final"]])]
    all_states = [to_string(state) for state in all_states]
    if deadstate:
        for symbol in symbols:
            transition[('X', symbol)] = 'X'
        all_states.append('X')
    dfa = {"start":str(start), "final":final, "states":all_states,
            "symbols":symbols, "transition":transition}
    #dfa = {"start":str(start),  "states":all_states,
    #        "symbols":symbols, "transition":transition}
    return dfa
            
                        


In [3]:
enfa = {'final': [3,4],
 'start': 1,
 'states': [1, 2, 3, 4],
 'symbols':['a','b'],
 'transition': {(1, 'a'): [2],
  (2, 'b'): [3,1],
  (3, 'a'): [3],
  (2, 'E'): [4],
  (4,'a'):[3],
  (4,'b'):[1]}}

nfa = enfa_to_nfa(enfa)
nfa

{'final': [3, 4],
 'start': 1,
 'states': [1, 2, 3, 4],
 'symbols': ['a', 'b'],
 'transition': {(1, 'a'): [2, 4],
  (2, 'a'): [3],
  (2, 'b'): [1, 3],
  (3, 'a'): [3],
  (4, 'a'): [3],
  (4, 'b'): [1]}}

In [4]:
nfa_to_dfa(nfa)

{'final': ['2,4', '3', '1,3', '2,3,4'],
 'start': '1',
 'states': ['1', '2,4', '3', '1,3', '2,3,4', 'X'],
 'symbols': ['a', 'b'],
 'transition': {('1', 'a'): '2,4',
  ('1', 'b'): 'X',
  ('1,3', 'a'): '2,3,4',
  ('1,3', 'b'): 'X',
  ('2,3,4', 'a'): '3',
  ('2,3,4', 'b'): '1,3',
  ('2,4', 'a'): '3',
  ('2,4', 'b'): '1,3',
  ('3', 'a'): '3',
  ('3', 'b'): 'X',
  ('X', 'a'): 'X',
  ('X', 'b'): 'X'}}